In [1]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from transformers import LayoutLMv3Processor, LayoutLMv3Model, LayoutLMv3Config
from PIL import Image
import pytesseract
import cv2
import numpy as np

# Model Definition
class DocumentForgeryDetector(nn.Module):
    def __init__(self, num_classes=5):
        super(DocumentForgeryDetector, self).__init__()
        self.resnet = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
        self.resnet = torch.nn.Sequential(*(list(self.resnet.children())[:-1]))

        self.layoutlm_config = LayoutLMv3Config.from_pretrained("microsoft/layoutlmv3-base")
        self.layoutlm = LayoutLMv3Model.from_pretrained("microsoft/layoutlmv3-base")

        self.resnet_features = 2048
        self.layoutlm_features = self.layoutlm_config.hidden_size

        self.fusion = nn.Sequential(
            nn.Linear(self.resnet_features + self.layoutlm_features, 512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, num_classes)
        )

    def forward(self, image_for_resnet, layoutlm_inputs):
        resnet_output = self.resnet(image_for_resnet)
        resnet_output = resnet_output.view(resnet_output.size(0), -1)
        layoutlm_output = self.layoutlm(**layoutlm_inputs).last_hidden_state[:, 0, :]
        combined_features = torch.cat((resnet_output, layoutlm_output), dim=1)
        return self.fusion(combined_features)

# OCR + Image/Text Processing
def extract_ocr_info(image_path):
    image = Image.open(image_path).convert("RGB")
    image_np = np.array(image)

    # OCR using pytesseract
    data = pytesseract.image_to_data(image_np, output_type=pytesseract.Output.DICT)

    words, boxes = [], []
    for i in range(len(data["text"])):
        if int(data["conf"][i]) > 0 and data["text"][i].strip():
            words.append(data["text"][i])
            (x, y, w, h) = (data["left"][i], data["top"][i], data["width"][i], data["height"][i])
            boxes.append([x, y, x + w, y + h])

    return image, words, boxes

# Inference Function
def predict_document(image_path, model_path='best_forgery_detector.pth', device='cpu'):
    # Load processor & model
    from transformers import LayoutLMv3ImageProcessor, AutoTokenizer
    image_processor = LayoutLMv3ImageProcessor.from_pretrained("microsoft/layoutlmv3-base", apply_ocr=False)
    tokenizer = AutoTokenizer.from_pretrained("microsoft/layoutlmv3-base")
    processor = LayoutLMv3Processor(image_processor=image_processor, tokenizer=tokenizer)

    model = DocumentForgeryDetector(num_classes=5)
    model.load_state_dict(torch.load(model_path, map_location=device, weights_only=True))
    model.to(device)
    model.eval()

    # Get OCR text + bounding boxes
    image, words, boxes = extract_ocr_info(image_path)

    # Prepare inputs for LayoutLMv3
    encoding = processor(image, words, boxes=boxes, return_tensors="pt", truncation=True, padding="max_length", max_length=512)
    layoutlm_inputs = {k: v.to(device) for k, v in encoding.items()}

    # Preprocess image for ResNet
    resnet_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    image_tensor = resnet_transform(image).unsqueeze(0).to(device)

    # Predict
    with torch.no_grad():
        outputs = model(image_tensor, layoutlm_inputs)
        probs = torch.softmax(outputs, dim=1)
        predicted = torch.argmax(probs, dim=1).item()
        confidence = probs[0, predicted].item() * 100  # Convert to percentage

    class_names = ['fraud1_copy_and_move', 'fraud2_face_morphing', 'fraud3_face_replacement', 'fraud4_combined', 'positive']
    return class_names[predicted], round(confidence, 2)


In [24]:
import os

def process_folder(folder_path, model_path='best_forgery_detector.pth', device='cpu'):
    results = []

    for filename in os.listdir(folder_path):
        if filename.lower().endswith('.png'):
            image_path = os.path.join(folder_path, filename)
            try:
                label, confidence = predict_document(image_path, model_path, device)
                print(f"[✓] {filename} --> {label} ({confidence:.2f}%)")
                results.append((filename, label, confidence))
            except Exception as e:
                print(f"[✗] Error processing {filename}: {e}")

    return results


# Example usage
folder_path = "D:/Projects/PE-2/review-3/Document_Forgery_Dataset_short/fraud1_copy_and_move"
all_results = process_folder(folder_path)


[✓] generated.photos_0139935.png --> fraud1_copy_and_move (63.33%)
[✓] generated.photos_0193017.png --> fraud1_copy_and_move (65.66%)
[✓] generated.photos_0210845.png --> fraud1_copy_and_move (58.76%)
[✓] generated.photos_v3_0000170.png --> fraud1_copy_and_move (64.74%)
[✓] generated.photos_v3_0000333.png --> fraud4_combined (69.66%)
[✓] generated.photos_v3_0000500.png --> fraud1_copy_and_move (70.65%)
[✓] generated.photos_v3_0000630.png --> fraud1_copy_and_move (67.96%)
[✓] generated.photos_v3_0002511.png --> fraud1_copy_and_move (57.46%)
[✓] generated.photos_v3_0002626.png --> fraud1_copy_and_move (61.37%)
[✓] generated.photos_v3_0002720.png --> fraud4_combined (53.32%)
[✓] generated.photos_v3_0003040.png --> fraud4_combined (51.76%)
[✓] generated.photos_v3_0003103.png --> fraud1_copy_and_move (64.30%)
[✓] generated.photos_v3_0003285.png --> fraud1_copy_and_move (61.34%)
[✓] generated.photos_v3_0003317.png --> fraud1_copy_and_move (59.82%)
[✓] generated.photos_v3_0003357.png --> frau

KeyboardInterrupt: 

In [23]:
import os

def process_folder(folder_path, model_path='best_forgery_detector.pth', device='cpu'):
    results = []

    for filename in os.listdir(folder_path):
        if filename.lower().endswith('.png'):
            image_path = os.path.join(folder_path, filename)
            try:
                label, confidence = predict_document(image_path, model_path, device)
                print(f"[✓] {filename} --> {label} ({confidence:.2f}%)")
                results.append((filename, label, confidence))
            except Exception as e:
                print(f"[✗] Error processing {filename}: {e}")

    return results


# Example usage
folder_path = "D:/Projects/PE-2/review-3/Document_Forgery_Dataset_short/fraud2_face_morphing"
all_results = process_folder(folder_path)


[✓] generated.photos_0139935.png --> positive (50.21%)
[✓] generated.photos_0193017.png --> positive (50.37%)
[✓] generated.photos_0210845.png --> positive (49.44%)
[✓] generated.photos_v3_0000170.png --> positive (50.29%)
[✓] generated.photos_v3_0000333.png --> positive (45.21%)
[✓] generated.photos_v3_0000500.png --> fraud2_face_morphing (50.16%)
[✓] generated.photos_v3_0000630.png --> positive (50.34%)
[✓] generated.photos_v3_0002511.png --> positive (50.06%)
[✓] generated.photos_v3_0002626.png --> positive (50.28%)
[✓] generated.photos_v3_0002720.png --> positive (48.75%)
[✓] generated.photos_v3_0003040.png --> positive (48.81%)
[✓] generated.photos_v3_0003103.png --> positive (50.37%)
[✓] generated.photos_v3_0003285.png --> positive (49.47%)
[✓] generated.photos_v3_0003317.png --> positive (48.82%)
[✓] generated.photos_v3_0003357.png --> positive (50.35%)
[✓] generated.photos_v3_0003687.png --> positive (49.91%)
[✓] generated.photos_v3_0004021.png --> fraud2_face_morphing (50.17%)

KeyboardInterrupt: 

In [22]:
import os

def process_folder(folder_path, model_path='best_forgery_detector.pth', device='cpu'):
    results = []

    for filename in os.listdir(folder_path):
        if filename.lower().endswith('.png'):
            image_path = os.path.join(folder_path, filename)
            try:
                label, confidence = predict_document(image_path, model_path, device)
                print(f"[✓] {filename} --> {label} ({confidence:.2f}%)")
                results.append((filename, label, confidence))
            except Exception as e:
                print(f"[✗] Error processing {filename}: {e}")

    return results


# Example usage
folder_path = "D:/Projects/PE-2/review-3/Document_Forgery_Dataset_short/fraud3_face_replacement"
all_results = process_folder(folder_path)


[✓] generated.photos_0139935.png --> fraud3_face_replacement (98.71%)
[✓] generated.photos_0193017.png --> fraud3_face_replacement (68.17%)
[✓] generated.photos_0210845.png --> fraud3_face_replacement (98.96%)
[✓] generated.photos_v3_0000170.png --> fraud3_face_replacement (99.15%)
[✓] generated.photos_v3_0000333.png --> fraud3_face_replacement (95.90%)
[✓] generated.photos_v3_0000500.png --> fraud3_face_replacement (98.87%)
[✓] generated.photos_v3_0000630.png --> fraud3_face_replacement (89.37%)
[✓] generated.photos_v3_0002511.png --> fraud3_face_replacement (98.93%)
[✓] generated.photos_v3_0002626.png --> fraud3_face_replacement (99.02%)
[✓] generated.photos_v3_0002720.png --> fraud3_face_replacement (68.67%)
[✓] generated.photos_v3_0003040.png --> fraud3_face_replacement (61.22%)
[✓] generated.photos_v3_0003103.png --> fraud3_face_replacement (99.01%)
[✓] generated.photos_v3_0003285.png --> fraud3_face_replacement (99.13%)
[✓] generated.photos_v3_0003317.png --> fraud3_face_replacem

KeyboardInterrupt: 

In [21]:
import os

def process_folder(folder_path, model_path='best_forgery_detector.pth', device='cpu'):
    results = []

    for filename in os.listdir(folder_path):
        if filename.lower().endswith('.png'):
            image_path = os.path.join(folder_path, filename)
            try:
                label, confidence = predict_document(image_path, model_path, device)
                print(f"[✓] {filename} --> {label} ({confidence:.2f}%)")
                results.append((filename, label, confidence))
            except Exception as e:
                print(f"[✗] Error processing {filename}: {e}")

    return results


# Example usage
folder_path = "D:/Projects/PE-2/review-3/Document_Forgery_Dataset_short/fraud4_combined"
all_results = process_folder(folder_path)


[✓] generated.photos_0139935.png --> fraud4_combined (92.25%)
[✓] generated.photos_0193017.png --> fraud1_copy_and_move (65.51%)
[✓] generated.photos_0210845.png --> fraud4_combined (94.83%)
[✓] generated.photos_v3_0000170.png --> fraud1_copy_and_move (65.28%)
[✓] generated.photos_v3_0000333.png --> fraud4_combined (70.80%)
[✓] generated.photos_v3_0000500.png --> fraud4_combined (89.16%)
[✓] generated.photos_v3_0000630.png --> fraud4_combined (94.42%)
[✓] generated.photos_v3_0002511.png --> fraud1_copy_and_move (55.35%)
[✓] generated.photos_v3_0002626.png --> fraud1_copy_and_move (63.82%)
[✓] generated.photos_v3_0002720.png --> fraud4_combined (51.42%)
[✓] generated.photos_v3_0003040.png --> fraud4_combined (52.86%)
[✓] generated.photos_v3_0003103.png --> fraud1_copy_and_move (63.39%)
[✓] generated.photos_v3_0003285.png --> fraud4_combined (68.65%)
[✓] generated.photos_v3_0003317.png --> fraud1_copy_and_move (52.45%)
[✓] generated.photos_v3_0003357.png --> fraud1_copy_and_move (62.35%)

KeyboardInterrupt: 

In [25]:
import os

def process_folder(folder_path, model_path='best_forgery_detector.pth', device='cpu'):
    results = []

    for filename in os.listdir(folder_path):
        if filename.lower().endswith('.png'):
            image_path = os.path.join(folder_path, filename)
            try:
                label, confidence = predict_document(image_path, model_path, device)
                print(f"[✓] {filename} --> {label} ({confidence:.2f}%)")
                results.append((filename, label, confidence))
            except Exception as e:
                print(f"[✗] Error processing {filename}: {e}")

    return results


# Example usage
folder_path = "D:/Projects/PE-2/review-3/Document_Forgery_Dataset_short/positive"
all_results = process_folder(folder_path)


[✓] generated.photos_0139935.png --> positive (50.40%)
[✓] generated.photos_0193017.png --> positive (50.23%)
[✓] generated.photos_0210845.png --> positive (48.50%)
[✓] generated.photos_v3_0000170.png --> positive (50.35%)
[✓] generated.photos_v3_0000333.png --> positive (45.58%)
[✓] generated.photos_v3_0000500.png --> fraud2_face_morphing (50.27%)
[✓] generated.photos_v3_0000630.png --> positive (50.36%)
[✓] generated.photos_v3_0002511.png --> positive (50.20%)
[✓] generated.photos_v3_0002626.png --> positive (50.27%)
[✓] generated.photos_v3_0002720.png --> positive (48.57%)
[✓] generated.photos_v3_0003040.png --> positive (48.91%)
[✓] generated.photos_v3_0003103.png --> positive (50.43%)
[✓] generated.photos_v3_0003285.png --> positive (50.34%)
[✓] generated.photos_v3_0003317.png --> positive (50.12%)
[✓] generated.photos_v3_0003357.png --> positive (50.26%)
[✓] generated.photos_v3_0003687.png --> positive (50.24%)
[✓] generated.photos_v3_0004021.png --> fraud2_face_morphing (50.17%)

KeyboardInterrupt: 

In [6]:
# Usage
doc_path = "D:/Projects/PE-2/review-3/Document_Forgery_Dataset_short/fraud2_face_morphing/generated.photos_v3_0000500.png"
label, confidence = predict_document(doc_path)
print(f"Predicted Class: {label}")
print(f"Confidence: {confidence:.2f}%")

Predicted Class: fraud2_face_morphing
Confidence: 50.16%
